In [24]:
import json
from scipy.spatial.transform import Rotation as R
import numpy as np

f = open('eastupsouth_pathdata.json')
d = json.load(f)
f = open('eastupsouth_metadata.json')
d2 = json.load(f)
d.update(d2)
d.keys()

dict_keys(['geoLocationAlignmentAttemptTimes', 'speechDataTime', 'PathID', 'speechData', 'geoLocationAlignmentAttempts', 'pathDataTime', 'PathDate', 'navigationDataTime', 'PathData', 'geoSpatialTransformTimes', 'userId', 'PathType', 'navigationData', 'geoSpatialTransforms', 'savedRouteGeospatialLocations', 'keypointData', 'isVoiceOverOn', 'cloudAnchorsForAlignment', 'trackingErrorTime', 'stateSequence', 'garAnchorTimestamps', 'settingsHistory', 'hasMap', 'trackingErrorPhase', 'garAnchorCameraWorldTransformsAndGeoSpatialData', 'garAnchors', 'routeId', 'trackingErrorData', 'stateSequenceTime', 'ARLoggerDataDir'])

In [33]:
def east_up_south_to_rot_mat(geo_spatial_transform):
    """Convert the East-Up-South to a rotation matrix.
    
       Note: that this function does not deal with the translation part of the transform
    
       Arguments:
           - geo_spatial_transform: the geo spatial transform captured by the ClewMaps logs
       Returns:
           - a 3 x 3 numpy array where each of the n geo spatial transforms has been
               converted to a rotation matrix
    """
    axis = np.array(geo_spatial_transform['eastUpSounth']['axis'])
    angle = geo_spatial_transform['eastUpSounth']['angle']
    return R.from_rotvec(axis*angle).as_matrix()

east_up_south = list(map(lambda x: east_up_south_to_rot_mat(x['geospatialTransform']),
                         d['garAnchorCameraWorldTransformsAndGeoSpatialData']))

Let's take a look at one of the returned elements and try to interpret it.  First

```
east_up_south[0]

array([[ 6.56424743e-02, -9.71035228e-01,  2.29742576e-01],
       [-9.62041716e-01, -4.67112738e-04,  2.72902031e-01],
       [-2.64890170e-01, -2.38935907e-01, -9.34207060e-01]])
```

We can interpret this by remembering the meaning of this matrix.  This matrix specifies the pose of the axes of the phone in the East-Up-South frame.  Each column of the matrix will tell us how a particular phone axis (e.g., the first column is the x-axis) can be represented by the East, Up, South axes.

```
east_up_south[0,:,0]

array([ 0.06564247, -0.96204172, -0.26489017])
```

We can see from the output above that the x-axis of the phone, which points down towards where the home button would be on the old iPhones, is mostly aligned with the negative up direction (meaning it's pointed down)...  That makes sense since I was holding the phone in portrait mode orientation.

```
east_up_south[0,:,2]

array([ 0.22974258,  0.27290203, -0.93420706])
```

From the above we can see that the z-axis of the phone, which points out of the screen is primarily composed of the negative south axis (this would mean the z-axis was pointing north or the rear camera was pointing south).